In [1]:
pip install -q tensorflow-model-optimization

     |████████████████████████████████| 174kB 2.9MB/s 


In [9]:
from keras.models import load_model
model=load_model('global_model_100.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                48        
_________________________________________________________________
dense_2 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_3 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 6,929
Trainable params: 6,929
Non-trainable params: 0
_________________________________________________________________


In [11]:
import tensorflow_model_optimization as tfmot
from tensorflow import keras

quantize_model = tfmot.quantization.keras.quantize_model
# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)
optimizer = keras.optimizers.Adam(lr=0.0001)
# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer=optimizer,loss='mean_squared_error')
q_aware_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
quantize_layer_3 (QuantizeLa (None, 2)                 3         
_________________________________________________________________
quant_dense_1 (QuantizeWrapp (None, 16)                53        
_________________________________________________________________
quant_dense_2 (QuantizeWrapp (None, 32)                549       
_________________________________________________________________
quant_dense_3 (QuantizeWrapp (None, 64)                2117      
_________________________________________________________________
quant_dense_4 (QuantizeWrapp (None, 64)                4165      
_________________________________________________________________
quant_dense_5 (QuantizeWrapp (None, 1)                 70        
Total params: 6,957
Trainable params: 6,929
Non-trainable params: 28
___________________________________________________

In [13]:
q_aware_model.save('Intel_quantize_aware_model.h5')

In [15]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()
with open('Intel_QAT.tflite', 'wb') as f:
  f.write(quantized_tflite_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/tmp3rjpu5pi/assets
